In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow import keras

import matplotlib.pyplot as plt

import zipfile as zp
import os
import glob
import random

In [ ]:
input_path = "/kaggle/input/aerial-cactus-identification"
## train
train_zpfile = zp.ZipFile(os.path.join(input_path, "train.zip"))
train_zpfile.extractall("/kaggle/working")
## test
test_zpfile = zp.ZipFile(os.path.join(input_path, "test.zip"))
test_zpfile.extractall("/kaggle/working")

In [ ]:
train_images = glob.glob('/kaggle/working/train/*')
train_images[0:4]

In [ ]:
from PIL import Image
img = Image.open('/kaggle/working/train/570ceff6e1c6eededfb4c256f9d6bfdd.jpg')
img.size

These images are so small.. I can't actual see anything.

In [ ]:
display(img)

In [ ]:
np.asarray(img).shape

In [ ]:
train_labels = pd.read_csv("/kaggle/input/aerial-cactus-identification/train.csv")
train_labels.head()

In [ ]:
train_labels['has_cactus'].value_counts().plot(kind='barh')

In [ ]:
os.makedirs('/kaggle/working/train/cactus_0')
os.makedirs('/kaggle/working/train/cactus_1')
os.makedirs('/kaggle/working/validation/cactus_0')
os.makedirs('/kaggle/working/validation/cactus_1')

In [ ]:
for i in range(train_labels.shape[0]):
    
    i_id = train_labels.loc[i, 'id']
    i_label = train_labels.loc[i, 'has_cactus']
    
    i_id_path = os.path.join('/kaggle/working/train/', i_id)
    train_path = "/kaggle/working/train"
    val_path = "/kaggle/working/validation"
    
    if i_label == 1:
        if random.uniform(0, 1) < 0.2:
            os.rename(i_id_path, os.path.join(val_path, 'cactus_1', i_id))
        else:            
            os.rename(i_id_path, os.path.join(train_path, 'cactus_1', i_id))
    else:
        if random.uniform(0, 1) < 0.2:
            os.rename(i_id_path, os.path.join(val_path, 'cactus_0', i_id))
        else:
            os.rename(i_id_path, os.path.join(train_path, 'cactus_0', i_id))

In [ ]:
train_images = glob.glob('/kaggle/working/train/*/*')
train_images[0:5]

In [ ]:
## image generators
train_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)
validation_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)
test_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)

## Create model
keras.backend.clear_session()
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(filters = 16, kernel_size = 3, padding = 'same', input_shape = (32, 32, 3), activation = 'elu', kernel_initializer = 'he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.MaxPooling2D(pool_size = 2, strides = 2, padding = 'same'))

for i in [32] * 1 + [64] * 1 + [128] * 1 + [256] * 1:
    model.add(keras.layers.Conv2D(filters = i, kernel_size = 3, padding = 'same', activation = 'elu', kernel_initializer = 'he_normal'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.MaxPooling2D(pool_size = 2, strides = 2, padding = 'same'))

model.add(keras.layers.Flatten())

for i in [712, 356]:
    model.add(keras.layers.Dense(i, activation = 'elu', kernel_initializer = 'he_normal'))

model.add(keras.layers.Dense(1, activation = 'sigmoid'))

## optimizer & model compile
optimizer = keras.optimizers.RMSprop(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer=optimizer)

## call back
callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(train_generator.flow_from_directory('/kaggle/working/train', 
                                                        target_size = (32, 32), 
                                                        class_mode = 'binary'),
                   validation_data=validation_generator.flow_from_directory('/kaggle/working/validation',
                                                                           target_size = (32, 32),
                                                                           class_mode = 'binary'),
                    epochs=100,
                    callbacks= callback,
                    steps_per_epoch = len(train_images)/10/16
                   )

In [ ]:
test_gen = test_generator.flow_from_directory('/kaggle/working',classes = ['test'], target_size=(32,32), class_mode=None,shuffle=None, batch_size=16)
ypred = model.predict_generator(test_gen)

In [ ]:
test_file_names = test_gen.filenames
ypred = ypred.flatten().tolist()

submission = pd.DataFrame({'id':range(1, len(test_file_names)+1), 'has_cactus':ypred})
submission.to_csv('submission.csv', index=False)